# Paramtypes explained

Internally in the SQLite database on disk, data are registered as being of one of (currently) four allowed types:

- `numeric`
- `array`
- `text`
- `complex`

This notebook seeks to exemplify when each type should be used, and how differently the `Measurement` object treats data of each type.

We start with necessary imports, and then initialising our database and creating an experiment.

In [1]:
import time
from pathlib import Path

import numpy as np

from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.instrument_drivers.mock_instruments import DummyInstrument
from qcodes.parameters import ArrayParameter

In [2]:
initialise_or_create_database_at(
    Path.cwd().parent / "example_output" / "paramtypes_explained.db"
)
exp = load_or_create_experiment("paramtypes", sample_name="not_available")

Let us, now, create two dummy instruments to be used in our experiment.

In [3]:
dac = DummyInstrument("dac", gates=["ch1", "ch2"])
SA = DummyInstrument("SA")

In [4]:
# some array-like data types


class Spectrum(ArrayParameter):
    def __init__(self, name, instrument):
        self.N = 7
        setpoints = (np.linspace(0, 1, self.N),)

        super().__init__(
            name=name,
            instrument=instrument,
            setpoints=setpoints,
            shape=(20,),
            label="Noisy spectrum",
            unit="V/sqrt(Hz)",
            setpoint_names=("Frequency",),
            setpoint_units=("Hz",),
        )

    def get_raw(self):
        return np.random.randn(self.N)


class MultiDimSpectrum(ArrayParameter):
    def __init__(self, name, instrument):
        self.start = 0
        self.stop = 1
        self.npts = (2, 5, 3)
        sp1 = np.linspace(self.start, self.stop, self.npts[0])
        sp2 = np.linspace(self.start, self.stop, self.npts[1])
        sp3 = np.linspace(self.start, self.stop, self.npts[2])
        setpoints = (
            sp1,
            np.tile(sp2, (len(sp1), 1)),
            np.tile(sp3, (len(sp1), len(sp2), 1)),
        )
        super().__init__(
            name=name,
            instrument=instrument,
            setpoints=setpoints,
            shape=(100, 50, 20),
            label="Flower Power Spectrum in 3D",
            unit="V/sqrt(Hz)",
            setpoint_names=("Frequency0", "Frequency1", "Frequency2"),
            setpoint_units=("Hz", "Other Hz", "Third Hz"),
        )

    def get_raw(self):
        a = self.npts[0]
        b = self.npts[1]
        c = self.npts[2]
        return np.reshape(np.arange(a * b * c), (a, b, c))


# a string-valued parameter


def dac1_too_high():
    return "Too high" if dac.ch1() > 5 else "OK"

Finally, we add our parameters to the dummy instruments:

In [5]:
dac.add_parameter("control", get_cmd=dac1_too_high)
SA.add_parameter("spectrum", parameter_class=Spectrum)
SA.add_parameter("spectrum3D", parameter_class=MultiDimSpectrum)

/opt/hostedtoolcache/Python/3.11.14/x64/lib/python3.11/site-packages/qcodes/instrument/instrument_base.py:188: QCoDeSDeprecationWarning: Parameter spectrum on instrument SA does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(
/opt/hostedtoolcache/Python/3.11.14/x64/lib/python3.11/site-packages/qcodes/instrument/instrument_base.py:188: QCoDeSDeprecationWarning: Parameter spectrum3D on instrument SA does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(


<__main__.MultiDimSpectrum: spectrum3D at 139678788842064>

## Numeric

The `numeric` datatype is simply a number. Data registered with this type are saved as individual numbers. This is the **default** datatype when registering parameters.

### Numeric example 1

In this example, all parameters get registered as `numeric` type. This entails that the array in unraveled and inserted point-by-point.

In [6]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum, setpoints=(dac.ch1,))

t0 = time.perf_counter()

with meas.run() as datasaver:
    for dac_v in np.linspace(0, 2, 5):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (SA.spectrum, SA.spectrum()))

t1 = time.perf_counter()

print(f"Finished run in {(t1 - t0):.3f} s")

dataset1 = datasaver.dataset

Starting experimental run with id: 1. 
Finished run in 0.026 s


The data may be retrieved using the `get_parameter_data` method. This function will bring back the data in a way that reflects the datastructure as it is stored.

In [7]:
dataset1.get_parameter_data()

{'SA_spectrum': {'SA_spectrum': array([[ 1.54849740e+00, -1.27669428e+00, -3.78733691e-01,
           9.30061917e-01, -4.08521112e-01,  5.36784315e-01,
          -9.89974377e-02],
         [ 1.53645590e+00,  7.04628569e-01, -9.63718384e-01,
          -2.05101640e-01, -2.16731125e-01,  8.09167305e-01,
          -8.35368355e-01],
         [-2.31773652e-01, -4.26180106e-01,  3.66647870e-01,
          -9.41882660e-01,  6.39268497e-01,  3.54521932e-01,
           1.48183926e+00],
         [-6.19216062e-01, -1.69048966e+00, -1.47259755e-01,
           4.35700225e-01, -2.42119158e-03, -1.23031649e-01,
          -1.23331638e+00],
         [ 9.53642170e-01,  7.39906797e-01, -1.77851026e-01,
           1.14668773e+00, -2.92133165e-01, -1.56409658e+00,
           2.46000837e+00]]),
  'dac_ch1': array([[0. , 0. , 0. , 0. , 0. , 0. , 0. ],
         [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
         [1. , 1. , 1. , 1. , 1. , 1. , 1. ],
         [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5],
         [2. , 2. , 2. 

## Array

The `array` paramtype stores data as binary blobs in the database. Insertion is faster (asymptotically **much** faster) this way, but the data are "dead" to SQL queries inside the database. Be informed that a BLOB in sqlite has a default max length limit set at 1 billion (1,000,000,000) bytes (for more information, refer to [Sqlite](https://sqlite.org/limits.html) docs).

### Array example 1

Let us repeat the above measurement, but this time using `array` paramtypes.

In [8]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum, setpoints=(dac.ch1,), paramtype="array")

t0 = time.perf_counter()

with meas.run() as datasaver:
    for dac_v in np.linspace(0, 2, 5):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (SA.spectrum, SA.spectrum()))

t1 = time.perf_counter()

print(f"Finished run in {(t1 - t0):.3f} s")

dataset2 = datasaver.dataset

Starting experimental run with id: 2. 
Finished run in 0.015 s


In [9]:
dataset2.get_parameter_data()

{'SA_spectrum': {'SA_spectrum': array([[-1.43236366, -0.31031334, -0.52699745,  0.04070555,  0.61355339,
           0.25179906, -3.46210655],
         [-1.26935102, -0.97383825,  1.65253027, -0.64195663, -0.2991338 ,
          -0.73266515,  0.05291552],
         [ 0.73984583,  2.19912832,  0.09298822,  0.17296226, -0.02889159,
           2.06719165, -1.02003757],
         [ 1.33925729, -1.39403915, -0.1042315 ,  0.31369145, -0.06674312,
          -1.59978522,  0.80283967],
         [ 0.3556757 , -0.44758938,  0.13966415,  0.98618381,  0.12914577,
          -0.39561614, -0.24979437]]),
  'dac_ch1': array([[0. , 0. , 0. , 0. , 0. , 0. , 0. ],
         [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
         [1. , 1. , 1. , 1. , 1. , 1. , 1. ],
         [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5],
         [2. , 2. , 2. , 2. , 2. , 2. , 2. ]]),
  'SA_Frequency': array([[0.        , 0.16666667, 0.33333333, 0.5       , 0.66666667,
          0.83333333, 1.        ],
         [0.        , 0.16666667, 0.33333333

### Array example 2

When storing multidimensional `array` data (think: Alazar cards), both `numeric` and `array` types can be used.

In [10]:
meas = Measurement(exp=exp)
meas.register_parameter(SA.spectrum3D, paramtype="array")

with meas.run() as datasaver:
    datasaver.add_result((SA.spectrum3D, SA.spectrum3D()))
dataset3 = datasaver.dataset

Starting experimental run with id: 3. 


The data come out the way we expect them to.

In [11]:
dataset3.get_parameter_data()

{'SA_spectrum3D': {'SA_spectrum3D': array([[[[ 0,  1,  2],
           [ 3,  4,  5],
           [ 6,  7,  8],
           [ 9, 10, 11],
           [12, 13, 14]],
  
          [[15, 16, 17],
           [18, 19, 20],
           [21, 22, 23],
           [24, 25, 26],
           [27, 28, 29]]]]),
  'SA_Frequency0': array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
  
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]]]),
  'SA_Frequency1': array([[[[0.  , 0.  , 0.  ],
           [0.25, 0.25, 0.25],
           [0.5 , 0.5 , 0.5 ],
           [0.75, 0.75, 0.75],
           [1.  , 1.  , 1.  ]],
  
          [[0.  , 0.  , 0.  ],
           [0.25, 0.25, 0.25],
           [0.5 , 0.5 , 0.5 ],
           [0.75, 0.75, 0.75],
           [1.  , 1.  , 1.  ]]]]),
  'SA_Frequency2': array([[[[0. , 0.5, 1. ],
           [0. , 0.5, 1. ],
           [0. , 0.5, 1. 

### Array example 3

For completeness, here, we provide an example where the multidimensional array has an auxiliary setpoint.

In [12]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(SA.spectrum3D, paramtype="array", setpoints=(dac.ch1,))

with meas.run() as datasaver:
    for dac_v in [3, 4, 5]:
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (SA.spectrum3D, SA.spectrum3D()))
dataset4 = datasaver.dataset

Starting experimental run with id: 4. 


In [13]:
dataset4.get_parameter_data()

{'SA_spectrum3D': {'SA_spectrum3D': array([[[[ 0,  1,  2],
           [ 3,  4,  5],
           [ 6,  7,  8],
           [ 9, 10, 11],
           [12, 13, 14]],
  
          [[15, 16, 17],
           [18, 19, 20],
           [21, 22, 23],
           [24, 25, 26],
           [27, 28, 29]]],
  
  
         [[[ 0,  1,  2],
           [ 3,  4,  5],
           [ 6,  7,  8],
           [ 9, 10, 11],
           [12, 13, 14]],
  
          [[15, 16, 17],
           [18, 19, 20],
           [21, 22, 23],
           [24, 25, 26],
           [27, 28, 29]]],
  
  
         [[[ 0,  1,  2],
           [ 3,  4,  5],
           [ 6,  7,  8],
           [ 9, 10, 11],
           [12, 13, 14]],
  
          [[15, 16, 17],
           [18, 19, 20],
           [21, 22, 23],
           [24, 25, 26],
           [27, 28, 29]]]]),
  'dac_ch1': array([[[[3., 3., 3.],
           [3., 3., 3.],
           [3., 3., 3.],
           [3., 3., 3.],
           [3., 3., 3.]],
  
          [[3., 3., 3.],
           [3., 3.,

## Text

Text is strings. Sometimes it may be useful to capture categorial data that is represented as string values, or a log message, or else.

Note that the `paramtype` setting is important. The datasaver will not allow to save `numeric` data for a parameter that was registered as `text`. The opposite it also true: the datasaver will not allow to save strings for a parameter what was registered as non-`text` (`numeric` or `array`).

In [14]:
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.control, setpoints=(dac.ch1,), paramtype="text")

with meas.run() as datasaver:
    for dac_v in np.linspace(4, 6, 10):
        dac.ch1(dac_v)
        datasaver.add_result((dac.ch1, dac_v), (dac.control, dac.control()))
dataset5 = datasaver.dataset

Starting experimental run with id: 5. 


In [15]:
dataset5.get_parameter_data()

{'dac_control': {'dac_control': array(['OK', 'OK', 'OK', 'OK', 'OK', 'Too high', 'Too high', 'Too high',
         'Too high', 'Too high'], dtype='<U8'),
  'dac_ch1': array([4.        , 4.22222222, 4.44444444, 4.66666667, 4.88888889,
         5.11111111, 5.33333333, 5.55555556, 5.77777778, 6.        ])}}